In [32]:
# import os
# from dotenv import load_dotenv

# load_dotenv()

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# MODEL = "gpt-3.5-turbo"
MODEL = "mistral"

In [33]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)
model.invoke("Tell me a joke")

' Why don\'t scientists trust atoms? Because they make up everything! \\*wink\\* (In case you were wondering, this is a play on the phrase "can\'t trust" and the fact that all matter is made of atoms.)'

In [34]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

" Why don't some numbers ever get invited to parties?\n\nBecause they always quit before the functions start! (Factors!)\n\nA classic math-related joke for you there! Enjoy your day."

In [42]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Allgemeine Geschäftsbedingungen SUS gmbh.pdf")
pages = loader.load_and_split()

In [43]:
from langchain.prompts import PromptTemplate

template = """" \
"Answer the question based on the context below. If you can't answer the question, reply "I don't know"." \
"" \
"Context: {context}" \
"" \
"Question: {question}" \
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'" "Answer the question based on the context below. If you can\'t answer the question, reply "I don\'t know"." "" "Context: Here is some context" "" "Question: Here is a question" '

In [44]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [45]:
retriever = vectorstore.as_retriever()

In [46]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

chain.invoke({"question": "Is this AGB normal?"})

" The provided document appears to be an Agreement with General Terms and Conditions (AGB) for a service provider (in this case, SUS GmbH). While I don't have the ability to evaluate whether these terms are common or not without comparing them to other similar agreements, some of the provisions might be considered unfavorable for the customer.\n   For instance, the AGB grants the provider the right to charge additional fees for handling inquiries (3. Zahlungsbedingungen, point 2), to hold sensitive customer data for internal analysis and marketing purposes (4. Datennutzung und Datenschutz, points 1 and 2), and to modify these terms at any time without prior notice (7. Schlussbestimmungen, point 1). Additionally, the customer's obligation to comply with the AGB even after termination of the contract (6. Vertragsdauer und Kündigung, point 3) may also be considered unusual.\n   To answer your question more precisely, it is difficult to say whether this particular AGB is normal or not with